In [8]:
%%time
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import json
import datetime
import lxml.html as LH

from math import sqrt, exp, log, erf    # For Black Scholes

from decimal import *
getcontext().prec = 5

from string import ascii_uppercase

# Interest rate (10-year bond yield)
inturl = requests.get("https://countryeconomy.com/bonds/india").content
intrate = float(LH.fromstring(inturl).find_class('numero')[0].text)/100

# Set today's days for DTE calculation
today = datetime.datetime.now().date()

# Get NSE Scrip expiry dates
expurl = "https://www.nseindia.com/live_market/dynaContent/live_watch/fomwatchsymbol.jsp?key=NIFTY&Fut_Opt=Futures"

exphtml = requests.get(expurl).content

fnotable = pd.read_html(exphtml, match='Expiry Date', header=0)[0]

fnoexpiry = fnotable['Expiry Date']

# Get the list of equity scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Remove VIX and NIFTY 
symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol

# Replace & by %26 for NSE
symbol.replace('&', '%26')

# Sort the symbols
symbol = symbol.sort_values(axis=0).reset_index(drop=True)

# Combine Expiry and Scrips to a new url DataFrame
urldf = pd.concat([symbol] * len(fnoexpiry), keys=fnoexpiry).reset_index().drop('level_1', axis=1)

# NSE URLs
urlbase = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
chainurl = urlbase + "option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol="
bandurl = urlbase + "get_quote/GetQuote.jsp?symbol="
volatilityurl = urlbase + "get_quote/GetQuoteFO.jsp?instrument=FUTSTK&underlying="


# Make a Dataframe of the NSE URLs
urldf['ChainURL'] = chainurl + urldf['Symbol'] + '&date=' + urldf['Expiry Date']
urldf['BandURL'] = bandurl + urldf['Symbol']
urldf['VolURL'] = volatilityurl + urldf['Symbol']

pd.set_option('display.max_colwidth', -1)                   # increase width to see all the data

failed = pd.DataFrame([], columns = ['Function', 'Symbol', 'Underlying', 'Expiry', 'Error'])   # Catch the symbols with errors

# Get the option chain function
def getchain(u, e, s, b, v, i): #u = url, e = expiry date, b = band, v = volatality, i = interest rate
    try:
        chainhtml = requests.get(u).content
        chain = pd.read_html(chainhtml)[1][:-1]  # read the first table and drop the total
        chain.columns=chain.columns.droplevel(0) # drop the first row of the header
        chain = chain.drop('Chart', 1)           # drop the charts

        # json for band
        bandhtml = requests.get(b).text  
        bandsoup = BeautifulSoup(bandhtml, 'html.parser')
        data = bandsoup.find(id='responseDiv').text.strip()
        d1 = json.loads(data)

        #json for volatility
        volhtml = requests.get(v).text  #remove IFCI when put in function!!
        volsoup = BeautifulSoup(volhtml, 'html.parser')
        data = volsoup.find(id='responseDiv').text.strip()
        d2 = json.loads(data)

        # Dividend rate
        divurl = "https://finance.google.com/finance?q=NSE:"+s
        page = requests.get(divurl)
        root = LH.fromstring(page.content)
        dividend = float(root.findall('.//table')[2].text_content().strip().split("\n")[2].split('/')[0])/100
                                     
        # match='Underlying Stock:' gives errors when table is not found
        underlyingtbl = pd.read_html(chainhtml, match='Underlying Stock:')[0][1]
        underlying = underlyingtbl.iloc[0]
        u = float(underlying.split(' ')[3])

        # parse for Lo52, Hi52, LoBand, HiBand, MarginRate, WhenLo52, WhenHi52
        Lo52 = float(d1['data'][0]['low52'].replace(',',''))
        Hi52 = float(d1['data'][0]['high52'].replace(',',''))
        WhenLo52 = d1['data'][0]['cm_adj_low_dt']
        WhenHi52 = d1['data'][0]['cm_adj_high_dt']
        LoBand = float(d1['data'][0]['pricebandlower'].replace(',',''))
        HiBand = float(d1['data'][0]['pricebandupper'].replace(',',''))
        MarginRate = float(d1['data'][0]['applicableMargin'])/100


        # get the Days-To-Expiry (DTE)
        exp = datetime.datetime.strptime(e, '%d%b%Y').date()
        DTE = (exp - today)

        # parse for daily volatility
        DailyVol = float(d2['data'][0]['dailyVolatility'])/100

        Volatility = float(d2['data'][0]['annualisedVolatility'])/100
        lot = float(d2['data'][0]['marketLot'].replace(',',''))
        # Volatility = DailyVol * sqrt(DTE.days)    #Volatility of remaining days-to-expiry


        chain['Underlying'] = u              # price of the underlying
        chain['Expiry'] = e                  # expiry date
        chain['Symbol'] = s                  # symbol
        chain['Lo52'] = Lo52
        chain['Hi52'] = Hi52
        chain['WhenLo52'] = datetime.datetime.strptime(WhenLo52, '%d-%b-%y')
        chain['WhenHi52'] = datetime.datetime.strptime(WhenHi52, '%d-%b-%y')
        chain['LoBand'] = LoBand
        chain['HiBand'] = HiBand
        chain['MarginRate'] = MarginRate
        chain['DailyVol']= DailyVol
        chain['DTE'] = DTE.days
        chain['Volatility'] = Volatility
        chain['Dividend'] = dividend
        chain['Lot'] = lot
        chain['IntRate'] = i

        return chain

# get the exceptions
    except (pd.core.groupby.DataError, AttributeError, ValueError, NameError, KeyError, IndexError) as err:
        failed.loc[len(failed)+1,['Function', 'Symbol', 'Underlying', 'Expiry', 'Error']] = \
                                 ['get_chain', s, u, e, repr(err)]

#Black & Scholes function
def calc_bs(symbol, undprice, strike, time, rate, sigma, divrate):

    try:
        #statistics
        sigTsquared = sqrt(Decimal(time)/365)*sigma
        edivT = exp((-divrate*time)/365)
        ert = exp((-rate*time)/365)
        d1 = (log(undprice*edivT/strike)+(rate+.5*(sigma**2))*time/365)/sigTsquared
        d2 = d1-sigTsquared
        Nd1 = (1+erf(d1/sqrt(2)))/2
        Nd2 = (1+erf(d2/sqrt(2)))/2
        iNd1 = (1+erf(-d1/sqrt(2)))/2
        iNd2 = (1+erf(-d2/sqrt(2)))/2

        #outputs
        callPrice = round(undprice*edivT*Nd1-strike*ert*Nd2, 2)
        putPrice = round(strike*ert*iNd2-undprice*edivT*iNd1, 2)
        prob = iNd2

        return callPrice, putPrice, prob
    
    except (pd.core.groupby.DataError, AttributeError, ValueError, NameError) as err:
        failed.loc[len(failed)+1,['Function', 'Symbol', 'Underlying', 'Error']] = \
                                 ['calc_bs', symbol, undprice, repr(err)]
    

###-----  BREVITY CODES -----###

# urldf = urldf[urldf.Symbol.str.startswith(('A', 'B', 'C', 'D', \
#                                            'D', 'E', 'F', 'G', 'H', \
#                                            'I', 'J', 'K', 'L', 'M', \
#                                            'N', 'O', 'P', 'R', 'S', \
#                                            'T', 'U', 'V', 'W', 'Y', 'Z'))]  # For brevity only first two symbols

# fnoexpiry = fnotable['Expiry Date'][0:1]          # Takes only the first date
### -------------------------###

# vectorize getchain function, pass arguments of urldf to it and rename output columns
v = np.vectorize(getchain)

output = pd.concat(v(urldf.ChainURL, urldf['Expiry Date'], urldf.Symbol, urldf.BandURL, urldf.VolURL, intrate)).reset_index(drop=True)


# Vectorize Black Scholes and get BSCall and BSPut
vbs = np.vectorize(calc_bs)
df =vbs(output.Symbol, output.Underlying, output['Strike Price'], 
        output.DTE.astype(float), output.IntRate, output.Volatility, output.Dividend)
df = pd.DataFrame(np.column_stack(df), columns = ['BSCall', 'BSPut', 'Probability'])

output = output.join(df)

# Rename columns to meaningful
chainheader = ['Call_OI', 'Call_OI_Change', 'Call_Volume', 'Call_IV', \
               'Call_LTP', 'Call_Net_Change', 'Call_BidQty', 'Call_BidPrice', 'Call_AskPrice', 'Call_AskQty', \
               'Strike', 'Put_BidQty', 'Put_BidPrice', 'Put_AskPrice', 'Put_AskQty', 'Put_Net_Change', \
               'Put_LTP', 'Put_IV', 'Put_Volume', 'Put_OI_Change', 'Put_OI', \
               'Underlying', 'Expiry', 'Symbol', 'Lo52', 'Hi52', 'WhenLo52', 'WhenHi52', 'LoBand', 'HiBand', \
               'MarginRate', 'DailyVol', 'DTE', 'Volatility', 'Dividend', 'Lot', 'IntRate', 'BSCall', 'BSPut', 'cProb']

output.columns = chainheader

#Convert non-numeric columns to numeric
cols = ['Call_OI', 'Call_OI_Change', 'Call_Volume', 'Call_IV', \
                     'Call_LTP', 'Call_Net_Change', 'Call_BidQty', 'Call_BidPrice', 'Call_AskPrice', \
                     'Call_AskQty', 'Strike', 'Put_BidQty', 'Put_BidPrice', 'Put_AskPrice', \
                     'Put_AskQty', 'Put_Net_Change', 'Put_LTP', 'Put_IV', 'Put_Volume', 'Put_OI_Change', 'Put_OI']

output[cols] = output[cols].apply(pd.to_numeric, errors='coerce')

#Fill the NaN with 0
output=output.fillna(0)

#Sort by Symbol and Strike - to show highs in the top and lows in the bottom (similar to IBKR)
output = output.sort_values(by=['Symbol', 'Strike'], ascending=[1,0]).reset_index(drop=True)

#Store the files
# x = r'NSEOptions'+symbol.iloc[0][:2]+'.xlsx'
x = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")+'_NSE.xlsx'
writer = pd.ExcelWriter(x)
output.to_excel(writer, 'Pass', index=False , freeze_panes=(1,1))
failed.to_excel(writer, 'Failed', index=False, freeze_panes=(1,1))
writer.save()

Wall time: 6.63 s
